# Container Installation

Source https://medium.com/@alexisrolland/rapid-api-development-tutorial-with-docker-postgresql-postgraphile-e387c1c73dd8

```bash
# Pulling
docker pull postgres:alpine

# Creating container with volume (provide your own local paths)
docker run --name psql -v /home/pi/data/postgresTrain -e POSTGRES_PASSWORD=<password_here> -p 5432:5432 -d postgres:alpine
```

## Testing command line (psql)

### Conection

How to connect to postgres

```bash
# For connection testing purposes
sudo apt-get install postgresql-client
psql -h 0.0.0.0 -p 5432 -U postgres #use password defined before
```

### Hello world

This is only to test everything is ok

```sql
CREATE DATABASE test_db;

\connect test_db;

CREATE SCHEMA test_schema;

CREATE TABLE test_schema.test_table (id SERIAL PRIMARY KEY,description TEXT);

INSERT INTO test_schema.test_table (description) VALUES ('Parent description 1');

SELECT * FROM test_schema.test_table;
```

### Second example

Now is a bit more complete example
```sql
CREATE SCHEMA test_schema2;

CREATE TABLE test_schema2.parent_table (
    id SERIAL PRIMARY KEY,
    name TEXT,
    description TEXT,
    created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

COMMENT ON TABLE test_schema2.parent_table IS 
    'Provide a description for your parent table.';

CREATE TABLE test_schema2.child_table (
    id SERIAL PRIMARY KEY,
    name TEXT,
    description TEXT,
    created_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    updated_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    parent_table_id INTEGER NOT NULL REFERENCES test_schema2.parent_table(id)
);

COMMENT ON TABLE test_schema2.child_table IS
'Provide a description for your child table.';

INSERT INTO test_schema2.parent_table (name, description) VALUES
('Parent name 1', 'Parent description 1'),
('Parent name 2', 'Parent description 2'),
('Parent name 3', 'Parent description 3');

INSERT INTO test_schema2.child_table (name, description, parent_table_id) VALUES
('Child name 1', 'Child description 1', 1),
('Child name 2', 'Child description 2', 2),
('Child name 3', 'Child description 3', 3);

SELECT A.id, A.name, A.description, B.id, B.name, B.description
FROM test_schema2.parent_table A
INNER JOIN test_schema2.child_table B ON A.id=B.parent_table_id;

```

> From here you can jump to run a pgadmin4 container and keep making queries (See the Postgres App Dev notebook). Keep going with more advanced Queries in a terminal can be a bit frustrating 

> It's recommended to play a bit more with the psql commands and options to gain experience in case something goes wrong at client side

## Psql commands

```bash
\dn -> list of schemas
\l -> list of databases
```

## Importing data



### Csv